# Train the model by preparing the dataset

In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import json
from src import preparedata
from src import modelarea
from src import trainarea

print(tf.__version__)
print(tf.config.list_physical_devices(
    device_type=None
))

params=json.load(open('params/params.json','r'))

2023-01-23 19:16:38.166626: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 19:16:38.413250: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-23 19:16:40.733067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dahala/miniconda3/envs/dlashok/lib/
2023-01-23 19:16:40.733174: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: lib

2.10.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
#prepare data
dataset=preparedata.readGPDData(params['dataprepinargs'])
dataset.preparedata()

  0%|          | 0/31 [00:00<?, ?it/s]

In [3]:
#prepare model
landslidehazard=modelarea.lhmodel(params['modelparam'])
landslidehazard.preparemodel()

2023-01-23 19:18:10.499196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 19:18:11.040056: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10404 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:b2:00.0, compute capability: 6.1


In [4]:
trainarea.trainmodel(landslidehazard.model,dataset.X_train,dataset.Y_train,params['trainparam'])
landslidehazard.model.save_weights('savedweights/final_modelfull_data.h5')

Epoch 1/20
1787/1790 [============================>.] - ETA: 0s - loss: -12462310.0000 - gpdmetric: 4481710.0000
Epoch 1: val_loss improved from inf to -15031245.00000, saving model to checkpoints/
INFO:tensorflow:Assets written to: checkpoints/assets
1790/1790 [==============================] - 25s 13ms/step - loss: -12463429.0000 - gpdmetric: 4481012.0000 - val_loss: -15031245.0000 - val_gpdmetric: 5299944.5000
Epoch 2/20
1789/1790 [============================>.] - ETA: 0s - loss: -15824234.0000 - gpdmetric: 5800638.5000
Epoch 2: val_loss improved from -15031245.00000 to -15909166.00000, saving model to checkpoints/
INFO:tensorflow:Assets written to: checkpoints/assets
1790/1790 [==============================] - 23s 13ms/step - loss: -15821153.0000 - gpdmetric: 5797993.5000 - val_loss: -15909166.0000 - val_gpdmetric: 5822623.0000
Epoch 3/20
1790/1790 [==============================] - ETA: 0s - loss: -16103245.0000 - gpdmetric: 5960246.0000
Epoch 3: val_loss improved from -15909166

In [5]:
landslidehazard.model.save_weights('savedweights/final_modelfull_data.h5')

In [29]:
landslidehazard.model.predict(dataset.X_train[:10,0])

AttributeError: 'readGPDData' object has no attribute 'X_train'

# Perform inference by calculating landslide hazrd

In [7]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import json
from src import preparedata
from src import modelarea
from src import inference
from src import savehazard

print(tf.__version__)
print(tf.config.list_physical_devices(
    device_type=None
))
params=json.load(open('params/params.json','r'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2.10.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [25]:
rp=20
dataset=preparedata.readGPDData(params['dataprepinargs'])
dataset.preparedatainference(rp)

  0%|          | 0/31 [00:00<?, ?it/s]

In [26]:
landslidehazard=modelarea.lhmodel(params['modelparam'])
landslidehazard.preparemodel()
haz=inference.inferenceLH(model=landslidehazard.model,model_weights='savedweights/final_modelfull_data.h5',xdata=dataset.Xinference,rp=rp)

2219/2219 [==============================] - 4s 2ms/step


In [29]:
savehazard.save_predicted_Haz(ids=dataset.InferenceID,prediction=haz,hazcol=f'haz_{rp}rp',sufile='Data/SlopeUnits/SlopeUnits_V3.shp',outfile=f'Data/Results/LsHaz{rp}_v2.shp')

In [27]:
haz.max()

0.22016518

In [28]:
haz.min()

0.11952155

In [12]:
import numpy as np 
np.exp(-1.7580993)

0.17237218046254701